Momentum investing is investing in stocks that have increased in price the most. 

Here, we will select the 50 stocks with highest price momentum. Then, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks. (From project 1)

In [14]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import yfinance as yf

In [25]:
# pandas read html
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# clean data
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = datetime.today().strftime('%Y-%m-%d')
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*2)

# stacking the data to make it easier to work with
# use future_stack=True to avoid the future warning
df = yf.download(tickers=symbols_list, 
                 start=start_date, 
                 end=end_date).stack(future_stack=True)

df

[*********************100%%**********************]  503 of 503 completed


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2022-08-01 A       131.583328  133.429993  135.229996  133.259995  133.509995   
           AAL      14.280000   14.280000   14.320000   13.520000   13.670000   
           AAPL    159.703171  161.509995  163.589996  160.889999  161.009995   
           ABBV    129.740509  140.220001  142.839996  139.149994  141.509995   
           ABNB    111.199997  111.199997  113.959999  107.480003  110.000000   
...                       ...         ...         ...         ...         ...   
2024-07-26 XYL     140.839996  140.839996  142.130005  137.820007  138.479996   
           YUM     128.050003  128.050003  129.039993  127.410004  127.690002   
           ZBH     111.290001  111.290001  112.279999  110.230003  110.790001   
           ZBRA    325.980011  325.980011  330.970001  323.000000  326.500000   
           ZTS     179.839996  179.839996  183.850006  179.259995  182.259995   

Price                  Volume  
Date       Ticker              
2022-08-01 A        1081700.0  
           AAL     32730800.0  
           AAPL    67829400.0  
           ABBV     8523900.0  
           ABNB     6019500.0  
...                       ...  
2024-07-26 XYL      1074100.0  
           YUM      1874400.0  
           ZBH      1399400.0  
           ZBRA      458700.0  
           ZTS      2437300.0  

[251500 rows x 6 columns]

In [57]:
# we want to pull price in 1 yr stock return
tickers_list = df.index.get_level_values(1).unique()
AllYF = yf.Tickers(' '.join(tickers_list)).tickers
# this is a replacement for year1ChangePercent
# yeet = AllYF['AAPL'].info['52WeekChange']

In [5]:
df_columns = ['Ticker', 'Stock Price', 'One-Year Price Return', 'Number of Shares to Buy']

data_list = [
    [
        ticker,
        values.info.get('previousClose', None), 
        values.info.get('52WeekChange', None),
        'N/A'
    ]
    for ticker, values in AllYF.items()
]

final_df = pd.DataFrame(data=data_list, columns=df_columns)
# 2:02

In [6]:
# inplace will directly modify final_df
final_df.sort_values(by='One-Year Price Return', ascending=False, inplace=True)
# get the top 50 results (50 highest returns)
final_df = final_df[:50]
# reset the index to start from 0
final_df.reset_index(drop=True, inplace=True)

Ngl, this is really lame. We will now create a more realistic momentum strategy with 1, 3, 6, and 12 month returns.

- High-quality m-stocks show "slow and steady" outperformance over long periods of time.
- Low-quality m-stocks might not show any momentum for a long time, then surge upwards.


In [67]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

# Assuming AllYF is your data source dictionary containing ticker and values
# AllYF = {...}

# Define the columns for the DataFrame
hqm_columns = [
    'Ticker',
    'Stock Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

# Initialize the DataFrame
hqm_df = pd.DataFrame(columns=hqm_columns)

# Define date ranges
today = datetime.now()
year1 = (today - relativedelta(years=1)).strftime('%Y-%m-%d')
month6 = (today - relativedelta(months=6)).strftime('%Y-%m-%d')
month3 = (today - relativedelta(months=3)).strftime('%Y-%m-%d')
month1 = (today - relativedelta(months=1)).strftime('%Y-%m-%d')

# Function to process each ticker
def process_ticker(ticker, values):
    try:
        stock_price = values.info.get('previousClose', None)
        one_year_return = (stock_price - values.history(start=year1, end=today)['Close'].to_numpy()[0]) / values.history(start=year1, end=today)['Close'].to_numpy()[0]
        six_month_return = (stock_price - values.history(start=month6, end=today)['Close'].to_numpy()[0]) / values.history(start=month6, end=today)['Close'].to_numpy()[0]
        three_month_return = (stock_price - values.history(start=month3, end=today)['Close'].to_numpy()[0]) / values.history(start=month3, end=today)['Close'].to_numpy()[0]
        one_month_return = (stock_price - values.history(start=month1, end=today)['Close'].to_numpy()[0]) / values.history(start=month1, end=today)['Close'].to_numpy()[0]

        return [
            ticker,
            stock_price,
            'N/A',
            one_year_return,
            'N/A',
            six_month_return,
            'N/A',
            three_month_return,
            'N/A',
            one_month_return,
            'N/A',
            'N/A'
        ]
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")
        return None

# Use ThreadPoolExecutor to process in parallel.
# the WITH statement is used to wrap the execution of a block of code, usually used when working with files, network connections, 
# or threads (what the case is here) that need to be cleaned up after use.
# files will be closed, even if an exception is raised at some point.
# a thread is a sequence of instructions that can be run in parallel with other threads.
# the process is simplified to just being called executor.

# max_workers is the number of threads to run in parallel
# futures is a list comprehension we iterate over each ticker and corresponding values in the AllYF dictionary.
# we submit the executor with the function that will be processed, along with standard for loop arguments.
# the for loop arguments 
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_ticker, ticker, values) for ticker, values in AllYF.items()]
    results = [future.result() for future in as_completed(futures)]

# Filter out any None results due to errors
results = [result for result in results if result is not None]

# Sort the results by ticker
results.sort(key=lambda x: x[0])

# Create the DataFrame
hqm_df = pd.DataFrame(data=results, columns=hqm_columns)


,Ticker,Stock Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.32,N/A,0.127700,N/A,0.037404,N/A,-0.014436,N/A,0.079865,N/A,N/A
1,AAL,10.62,N/A,-0.365970,N/A,-0.289157,N/A,-0.240343,N/A,-0.038043,N/A,N/A
2,AAPL,217.96,N/A,0.115386,N/A,0.139802,N/A,0.257958,N/A,0.005582,N/A,N/A
3,ABBV,185.16,N/A,0.285693,N/A,0.150668,N/A,0.156890,N/A,0.096795,N/A,N/A
4,ABNB,140.10,N/A,-0.079440,N/A,-0.082515,N/A,-0.136518,N/A,-0.076040,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,140.84,N/A,0.249135,N/A,0.242414,N/A,0.062222,N/A,0.070782,N/A,N/A
499,YUM,128.05,N/A,-0.069877,N/A,-0.019150,N/A,-0.105734,N/A,-0.012722,N/A,N/A
500,ZBH,111.29,N/A,-0.194426,N/A,-0.094614,N/A,-0.085613,N/A,0.050302,N/A,N/A
501,ZBRA,325.98,N/A,0.058514,N/A,0.273708,N/A,0.086564,N/A,0.064564,N/A,N/A


In [78]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_df[change_col], hqm_df.loc[row, change_col]) /100


,Ticker,Stock Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MHK,160.71,124,0.511285,0.72,0.553654,0.96,0.357004,0.94,0.462329,1.0,97.862823
1,MMM,127.16,157,0.439444,0.62,0.615306,0.98,0.389357,0.98,0.263890,0.94,97.117296
2,UHS,213.69,93,0.537781,0.8,0.362384,0.82,0.263541,0.86,0.172832,0.76,95.874751
3,TYL,590.65,33,0.489171,0.7,0.353429,0.78,0.269669,0.88,0.182435,0.82,95.725646
4,KKR,118.51,168,1.013139,1.0,0.364361,0.84,0.245941,0.8,0.133958,0.56,95.328032
5,CFG,43.23,462,0.416722,0.56,0.293007,0.62,0.250868,0.82,0.206531,0.88,93.737575
6,IRM,98.50,203,0.662289,0.92,0.477328,0.92,0.272705,0.9,0.098227,0.3,93.290258
7,GRMN,177.94,112,0.718308,0.94,0.435135,0.9,0.231440,0.76,0.104531,0.38,93.141153
8,DHI,176.94,113,0.404942,0.52,0.267305,0.5,0.214724,0.7,0.291344,0.98,92.793241
9,CBRE,110.55,180,0.326972,0.26,0.258109,0.46,0.257107,0.84,0.285914,0.96,91.302187


In [79]:
from statistics import mean
# the HQM score looks at how much the return is for 1, 3, 6, and 12 months. It will then take the mean to get the final score.
# this is ideal for looking at stocks that have been consistently performing well over the past year.
# this will not account for drops in the stock price in the first few months.
for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

,Ticker,Stock Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MHK,160.71,124,0.511285,0.72,0.553654,0.96,0.357004,0.94,0.462329,1.0,0.905
1,MMM,127.16,157,0.439444,0.62,0.615306,0.98,0.389357,0.98,0.263890,0.94,0.88
2,UHS,213.69,93,0.537781,0.8,0.362384,0.82,0.263541,0.86,0.172832,0.76,0.81
3,TYL,590.65,33,0.489171,0.7,0.353429,0.78,0.269669,0.88,0.182435,0.82,0.795
4,KKR,118.51,168,1.013139,1.0,0.364361,0.84,0.245941,0.8,0.133958,0.56,0.8
5,CFG,43.23,462,0.416722,0.56,0.293007,0.62,0.250868,0.82,0.206531,0.88,0.72
6,IRM,98.50,203,0.662289,0.92,0.477328,0.92,0.272705,0.9,0.098227,0.3,0.76
7,GRMN,177.94,112,0.718308,0.94,0.435135,0.9,0.231440,0.76,0.104531,0.38,0.745
8,DHI,176.94,113,0.404942,0.52,0.267305,0.5,0.214724,0.7,0.291344,0.98,0.675
9,CBRE,110.55,180,0.326972,0.26,0.258109,0.46,0.257107,0.84,0.285914,0.96,0.63


In [80]:
# unfortunately, these are still going to be equal weight. We can later adjust this, but that's beyond the scope of this project.

hqm_df.sort_values('HQM Score', ascending=False, inplace=True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(drop=True, inplace=True)

C:\Users\mcubi\AppData\Local\Temp\ipykernel_28344\1462419219.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_df.sort_values('HQM Score', ascending=False, inplace=True)


,Ticker,Stock Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MHK,160.71,124,0.511285,0.72,0.553654,0.96,0.357004,0.94,0.462329,1.0,0.905
1,MMM,127.16,157,0.439444,0.62,0.615306,0.98,0.389357,0.98,0.263890,0.94,0.88
2,UHS,213.69,93,0.537781,0.8,0.362384,0.82,0.263541,0.86,0.172832,0.76,0.81
3,KKR,118.51,168,1.013139,1.0,0.364361,0.84,0.245941,0.8,0.133958,0.56,0.8
4,TYL,590.65,33,0.489171,0.7,0.353429,0.78,0.269669,0.88,0.182435,0.82,0.795
5,IRM,98.50,203,0.662289,0.92,0.477328,0.92,0.272705,0.9,0.098227,0.3,0.76
6,GRMN,177.94,112,0.718308,0.94,0.435135,0.9,0.231440,0.76,0.104531,0.38,0.745
7,CFG,43.23,462,0.416722,0.56,0.293007,0.62,0.250868,0.82,0.206531,0.88,0.72
8,FICO,1605.94,12,0.916465,0.98,0.313781,0.7,0.404948,1.0,0.073123,0.12,0.7
9,DHI,176.94,113,0.404942,0.52,0.267305,0.5,0.214724,0.7,0.291344,0.98,0.675


In [81]:
from CustomFunctions import get_int

portfolio_size = get_int()
position_size = portfolio_size / len(hqm_df.index)
# / returns a float, // returns an int
for i in hqm_df.index:
    hqm_df.loc[i, "Number of Shares to Buy"] = math.floor(position_size / hqm_df.loc[i, "Stock Price"])

hqm_df

,Ticker,Stock Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MHK,160.71,124,0.511285,0.72,0.553654,0.96,0.357004,0.94,0.462329,1.0,0.905
1,MMM,127.16,157,0.439444,0.62,0.615306,0.98,0.389357,0.98,0.263890,0.94,0.88
2,UHS,213.69,93,0.537781,0.8,0.362384,0.82,0.263541,0.86,0.172832,0.76,0.81
3,KKR,118.51,168,1.013139,1.0,0.364361,0.84,0.245941,0.8,0.133958,0.56,0.8
4,TYL,590.65,33,0.489171,0.7,0.353429,0.78,0.269669,0.88,0.182435,0.82,0.795
5,IRM,98.50,203,0.662289,0.92,0.477328,0.92,0.272705,0.9,0.098227,0.3,0.76
6,GRMN,177.94,112,0.718308,0.94,0.435135,0.9,0.231440,0.76,0.104531,0.38,0.745
7,CFG,43.23,462,0.416722,0.56,0.293007,0.62,0.250868,0.82,0.206531,0.88,0.72
8,FICO,1605.94,12,0.916465,0.98,0.313781,0.7,0.404948,1.0,0.073123,0.12,0.7
9,DHI,176.94,113,0.404942,0.52,0.267305,0.5,0.214724,0.7,0.291344,0.98,0.675


In [82]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name='Momentum Strategy', index=False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['One-Year Price Return', percent_format],
    'E': ['One-Year Return Percentile', percent_format],
    'F': ['Six-Month Price Return', percent_format],
    'G': ['Six-Month Return Percentile', percent_format],
    'H': ['Three-Month Price Return', percent_format],
    'I': ['Three-Month Return Percentile', percent_format],
    'J': ['One-Month Price Return', percent_format],
    'K': ['One-Month Return Percentile', percent_format],
    'L': ['HQM Score', percent_format]
}

for column, format in column_formats.items():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, format[1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', format[0], format[1])

writer.close()